In [42]:
import tensorflow as tf
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
#from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, TFBertModel
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler, ModelCheckpoint, TensorBoard
from seqeval.metrics import classification_report
import shutil
import pickle
from seqeval.metrics import classification_report
import os
import json

In [2]:
BERT_MODEL = 'bert-base-uncased'
MAX_SEQ_LEN = 50

In [3]:
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL)

In [4]:
class JointIntentAndSlotFillingModel(tf.keras.Model):

    def __init__(self, total_intent_no=None, total_slot_no=None,
                 model_name=BERT_MODEL, dropout_prob=0.1):
        super().__init__()
        self.bert = TFBertModel.from_pretrained(model_name)
        self.dropout = Dropout(dropout_prob)
        self.intent_classifier = Dense(total_intent_no, activation='softmax')
        self.slot_classifier = Dense(total_slot_no, activation='softmax')

    def call(self, inputs, **kwargs):
        bert_output = self.bert(inputs)

        sequence_output = self.dropout(bert_output[0])
        slots_predicted = self.slot_classifier(sequence_output)

        pooled_output = self.dropout(bert_output[1])
        intent_predicted = self.intent_classifier(pooled_output)

        return slots_predicted, intent_predicted

In [5]:
with open(r'/home/CE/lkiefer/BERT/rule_based_chatbot/joint_model/intent_label_encoder.pkl','rb') as fp:
  intent_le = pickle.load(fp)

In [6]:
with open(r'/home/CE/lkiefer/BERT/rule_based_chatbot/joint_model/seq_out_index_word.pkl', 'rb') as fp:
  index_to_word = pickle.load(fp)

In [7]:
BERT_MODEL = 'bert-base-uncased'
#tokenizer = BertTokenizer.from_pretrained(BERT_MODEL)

#load model checkpoints
checkpoint_dirs = r'/home/CE/lkiefer/BERT/rule_based_chatbot/checkpoints'
latest = tf.train.latest_checkpoint(checkpoint_dirs)
#latest = r'/home/CE/lkiefer/BERT/rule_based_chatbot/checkpoints/joint_model_weights_0.49.ckpt.data-00000-of-00001'
print(latest)

model = JointIntentAndSlotFillingModel(
    total_intent_no=3, total_slot_no=17,dropout_prob=0.1)

#model.built = True

model.load_weights(latest)

opt = Adam(learning_rate=3e-5, epsilon=1e-08)
losses = [SparseCategoricalCrossentropy(from_logits=False),
          SparseCategoricalCrossentropy(from_logits=False)]
metrics = [SparseCategoricalAccuracy('accuracy')]
model.compile(optimizer=opt, loss=losses, metrics = metrics)

/home/CE/lkiefer/BERT/rule_based_chatbot/checkpoints/joint_model_weights_0.49.ckpt


2023-08-31 16:10:56.411295: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you a

In [8]:
def detokenize(slots):

  result = {}
  current_key = ''
  current_value = None

  for key, value in slots.items():
      if key.startswith('##'):
          current_key += key[2:]
      else:
          if current_key:
              result[current_key] = current_value
          current_key = key
          current_value = value

  if current_key:
      result[current_key] = current_value

  return result

In [9]:
def show_predictions(text):

  tokenized_sent = tokenizer.encode(text)

  predicted_slots, predicted_intents = model.predict([tokenized_sent])

  intent = intent_le.inverse_transform([np.argmax(predicted_intents)])
  #print("="*5, "INTENT", "="*5)
  #print(intent)

  slots = np.argmax(predicted_slots, axis=-1)

  slots = [index_to_word[w_idx] for w_idx in slots[0]]

  #print("\n")
  #print("="*5, "SLOTS","="*5)

  slot_dict = {}

  for w,l in zip(tokenizer.tokenize(text),slots[1:-1]):
    #print(w,  l)
    slot_dict[w] = l

  slot_dict = detokenize(slot_dict)

  return intent, slot_dict

In [10]:
def diff_question():
    user = input("Please ask a different question.")
    return user


In [11]:
 def full_slots(slot_dict):   
    full_slots = {}
    current_key = None
    current_value = None

    for key, value in slot_dict.items():
        if value.startswith('B-'):
            if current_key:
                full_slots[current_key] = current_value
            current_key = key
            current_value = value[2:]  # Remove the 'B-' prefix
        elif value.startswith('I-'):
            if current_key:
                current_key += ' ' + key
                current_value = value[2:]  # Remove the 'I-' prefix
        else:
            if current_key:
                full_slots[current_key] = current_value
                current_key = None
                current_value = None
            full_slots[key] = value

    # Handle the last entry
    if current_key:
        full_slots[current_key] = current_value

    return full_slots

In [16]:
test = {'i': 'O', 'need': 'B-help', 'help': 'I-help', 'taking': 'O', 'care': 'O', 'of': 'O', 'my': 'O', 'monstera': 'B-PlantName', 'plant': 'I-PlantName', '.': 'PAD'}

full_slots(test)

{'i': 'O',
 'need help': 'help',
 'taking': 'O',
 'care': 'O',
 'of': 'O',
 'my': 'O',
 'monstera plant': 'PlantName',
 '.': 'PAD'}

In [37]:
def conversation(user, intent, slot_dict):
    print(slot_dict)
    if intent != 'plant_caretaking_advice':
        user = diff_question()
        intent, slots = show_predictions(user)
        conversation(user, intent, slots)
    else:
        slots = full_slots(slot_dict)
        plant_name = None
        for k,v in slots.items():
            if v == "PlantName" in slots.values():
                plant_name = k
            
    if plant_name == None:
        plant_name = input("Please state the name of your plant!")
        
    print("Plant name: ", plant_name)
    print("slots", slots)
    return slots

In [51]:
def search_plant(folder_path, search_string):
    for filename in os.listdir(folder_path):
        if filename.endswith(".json") and search_string == filename:
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r') as json_file:
                content = json.load(json_file)
                maintenance_info = content["maintenance"]
                print("maintenance: ")
                for key, value in maintenance_info.items():
                    print(f"{key}: {value}")
                #print(f"Content of {filename}:")
                #print(json.dumps(content, indent=4))
                #print("-" * 40)

# Replace these with your desired folder path and search string
folder_path = "/home/CE/lkiefer/BERT/data/plant-database-master/json"
search_string = "actaea racemosa.json"

search_plant(folder_path, search_string)

maintenance: 
size: Diameter ≥ 20 cm, height 20-30 cm
soil: Peat or soil with specific nutrients
sunlight: Like half shade, may grow in places with bright scattered light
watering: Water thoroughly when soil is dry
fertilization: Mix proper amount of slow-release fertilizers when planting, apply liquid fertilizers 1-2 times monthly
pruning: Remove dead and diseased branches timely


In [39]:
user = input("Hi! I am PlantPal. How can I help you?")
intent, slots = show_predictions(user)
all_slots = conversation(user, intent, slots)

1/1 [==============================] - 0s 77ms/step
{'i': 'O', 'need': 'O', 'help': 'O', 'taking': 'O', 'care': 'I-AdviceType', 'of': 'O', 'my': 'O', 'abelia': 'O', '.': 'PAD'}
Plant name:  abelia.
slots {'i': 'O', 'need': 'O', 'help': 'O', 'taking': 'O', 'of': 'O', 'my': 'O', 'abelia': 'O', '.': 'PAD'}


In [33]:
print(all_slots)
# I need help taking care of my pothos plant.

{'i': 'O', 'need': 'O', 'help': 'O', 'taking': 'O', 'care': 'O', 'of': 'O', 'my': 'O', 'pothos plant': 'PlantName', '.': 'PAD'}
